# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('./data/home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt

for attr in ['sqft_living', 'sqft_lot']:
  sales[attr + '_sqrt'] = sales[attr].apply(sqrt)
  
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 

for attr in ['bedrooms', 'floors']:
  sales[attr + '_square'] = sales[attr] * sales[attr]

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
All_Features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=All_Features,
                                                validation_set=None, 
                                                l2_penalty=0., 
                                                l1_penalty=1e10,
                                                verbose=False)

Find what features had non-zero weight.

In [6]:
## non zero weight
non_zero_weight = model_all.coefficients[model_all.coefficients['value'] != 0]
non_zero_weight

name,index,value,stderr
(intercept),None,274873.0559504957,None
bathrooms,None,8468.531086910107,None
sqft_living,None,24.420720982445445,None
sqft_living_sqrt,None,350.0605533860561,None
grade,None,842.0680348976241,None
sqft_above,None,20.024722417091116,None


In [7]:
## zero weight
eps = 1e-6
zero_weight = model_all.coefficients[model_all.coefficients['value'].abs() <= eps]

In [8]:
## number of coefficients(weights) / number of non-zero weights / number of zero weights
len(model_all.coefficients), len(non_zero_weight), len(zero_weight)

(18, 6, 12)

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***Quiz Question***:
According to this list of weights, which of the features have been chosen? 

In [9]:
## Answer: features chosen (intercept id at index 0)
non_zero_weight['name'][1:]

dtype: str
Rows: 5
['bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']

In [10]:
model_all.coefficients[['name', 'value']]

name,value
(intercept),274873.0559504957
bedrooms,0.0
bedrooms_square,0.0
bathrooms,8468.531086910107
sqft_living,24.420720982445445
sqft_living_sqrt,350.0605533860561
sqft_lot,0.0
sqft_lot_sqrt,0.0
floors,0.0
floors_square,0.0


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_validation, testing) = sales.random_split(.9, seed=1)         # initial train/test split
(training, validation) = training_validation.random_split(0.5, seed=1)  # split (again)  training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
import numpy as np

In [13]:
def get_rss(model, data, y):
    preds = model.predict(data)   # First get the predictions
    diff = y - preds              # Then compute the residuals/errors
    rss = (diff * diff).sum()     # Then square and add them up
    return rss

In [14]:
def find_bestl1(l1_range=np.logspace(1, 7, num=13), target='price', verbose=False):
  best_l1, min_rss, best_model = None, None, None
  target='price'
  l1_nnz = []
  
  for l1p in l1_range:
    model_ = turicreate.linear_regression.create(training, target=target, features=All_Features,
                                                 l2_penalty=0., 
                                                 l1_penalty=l1p,
                                                 validation_set=None, 
                                                 verbose=False)
    rss = get_rss(model_, validation, validation[target])
    nnz_ = model_.coefficients['value'].nnz()
    l1_nnz.append((l1p, nnz_))
    #
    if verbose:
      print(f"\tcurrent rss: {rss:15.2f} / l1_penalty: {l1p:7.1f} / nnz: {nnz_:2d}")
    #
    if min_rss is None or rss <= min_rss:
      min_rss, best_l1 = rss, l1p
      best_model = model_
  #
  # non zero:
  nnz = best_model.coefficients['value'].nnz()
  return (best_l1, min_rss, best_model, nnz, l1_nnz)

In [15]:
(best_l1, min_rss, best_model, nnz, _) = find_bestl1()
print(f"min_rss: {min_rss:15.2f} / l1_penalty: {best_l1:7.1f}")

min_rss: 625766285142460.38 / l1_penalty:    10.0


***Quiz Qestion.*** What was the best value for the `l1_penalty`?

In [16]:
best_l1

10.0

***Quiz Question***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [17]:
# len(best_model.coefficients[best_model.coefficients['value'] > 0.])
nnz

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [20]:
(_best_l1, _min_rss, _best_model, _nnz, l1_nnz) = find_bestl1(l1_range=l1_penalty_values)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [21]:
l1_penalty_max = list(filter(lambda t: t[1] < max_nonzeros, sorted(l1_nnz, key=lambda t: t[0], reverse=False)))[0]
l1_penalty_min = list(filter(lambda t: t[1] > max_nonzeros, sorted(l1_nnz, key=lambda t: t[0], reverse=True)))[0]

l1_penalty_min

(2976351441.6313133, 10)

In [22]:
l1_penalty_max

(3792690190.7322536, 6)

***Quiz Qestion.***  What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 
  - cf. above cells

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [23]:
l1_penalty_values = np.linspace(l1_penalty_min[0], l1_penalty_max[0], 20)

* For `l1_penalty` in `np.linspace(l1_penalty_min, l1_penalty_max, 20)`:
    * Fit a regression model with a given `l1_penalty` on *train* data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the *validation* set

Find the model that the lowest RSS on the *validation* set and has **sparsity *equal* to `max_nonzeros`**.

In [24]:
(best_l1, min_rss, best_model, nnz, l1_nnz) = find_bestl1(l1_range=l1_penalty_values)
print(f"[not taking into account max_non_zeros] min_rss: {min_rss:15.2f} / l1_penalty: {best_l1:7.1f}")

[not taking into account max_non_zeros] min_rss: 966925692362085.00 / l1_penalty: 2976351441.6


***Quiz Questions***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the *validation* set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [25]:
## 1.
best_l1 = min(
  map(lambda t: t[0], filter(lambda t: t[1] == max_nonzeros, l1_nnz))
)
best_l1

3448968612.163437

In [26]:
## 2.
nzw = best_model.coefficients[best_model.coefficients['value'] != 0]
nzw['name'][1:]

dtype: str
Rows: 9
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'floors', 'view', 'grade', 'sqft_above', 'sqft_basement']